In [0]:
X  date                    datetime64[ns]
X  category                        object
X  source                          object
X  rating                         float64
X  review_text                     object
X  sentiment                       object
  quality_score                  float64
  price_score                    float64
  service_score                  float64
  innovation_score               float64
  design_score                   float64
  market_share                   float64 random numbers now
  competitive_position           float64
X  month                            int32
X  quarter                          int32
  seasonal_rating                float64
  review_category                 object
X  review_length                    int64
  response_time                  float64
  resolution_time                float64

In [0]:
%sql

CREATE OR REPLACE VIEW retail_cpg_demo.brand_manager.vw_brand_insights AS

WITH top_brands AS (

  SELECT
    main_category,
    store,
    avg_price,
    num_products
  FROM (
    SELECT
      main_category,
      store,
      avg(price) as avg_price,
      count(distinct parent_asin) as num_products,
      row_number() over (partition by main_category order by main_category, sum(rating_number) desc) AS brand_num
    FROM retail_cpg_demo.brand_manager.product_details
    WHERE main_category IN (
      'Cell Phones & Accessories',
      'Toys & Games',
      'All Electronics',
      'Musical Instruments',
      'Home Audio & Theater',
      'Industrial & Scientific',
      'Sports & Outdoors',
      'All Beauty',
      'Tools & Home Improvement',
      'Office Products',
      'Automotive',
      'Baby'
    )
    AND store IS NOT NULL
    GROUP BY
    main_category, store
  )
  WHERE brand_num <= 20
)

SELECT
  r.date,
  d.main_category AS category,
  d.store AS brand,
  r.rating,
  r.text AS review_text,
  get_json_object(r.structured_review, '$.sentiment') AS sentiment,
  b.avg_price AS avg_brand_price,
  b.num_products AS num_brand_products
FROM retail_cpg_demo.brand_manager.product_reviews_structured r
INNER JOIN retail_cpg_demo.brand_manager.product_details d
  ON r.parent_asin = d.parent_asin
INNER JOIN top_brands b
  ON d.main_category = b.main_category
  AND d.store = b.store
WHERE d.main_category IS NOT NULL
  

In [0]:
%sql

SELECT count(1) from retail_cpg_demo.brand_manager.vw_brand_insights

In [0]:
%sql
select * from retail_cpg_demo.brand_manager.product_reviews_structured
limit 1000